In [1]:
from transformers import BertTokenizer, BertForMaskedLM, BertForSequenceClassification
import torch

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# # retrieve index of [MASK]
# mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

# predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
# tokenizer.decode(predicted_token_id)

# labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# # mask labels of non-[MASK] tokens
# labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

# outputs = model(**inputs, labels=labels)
# round(outputs.loss.item(), 2)


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", problem_type="multi_label_classification", num_labels=3)
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
model.max_seq_len

AttributeError: 'BertForSequenceClassification' object has no attribute 'max_seq_len'

In [3]:
import yaml
from easydict import EasyDict
from optimum.graphcore import IPUSeq2SeqTrainingArguments as Seq2SeqTrainingArguments

In [4]:
argus = {}
with open('configurations/electra.yaml') as f:
    hparams = yaml.load_all(f, Loader=yaml.FullLoader)
    for argu in hparams:
        argus[list(argu.keys())[0]]=list(argu.values())[0]

In [5]:
model_args, data_args, training_args = argus['ModelArguments'], argus['DataTrainingArguments'], argus['IPUSeq2SeqTrainingArguments']
model_args, data_args = EasyDict(model_args), EasyDict(data_args)
training_args = Seq2SeqTrainingArguments(**training_args)

In [6]:
from optimum.graphcore.models.bert.modeling_bert import PipelinedBertForMaskedLM, PipelinedBertForSequenceClassification
from optimum.graphcore import IPUConfig


In [7]:
ipu_config = IPUConfig.from_pretrained('configurations/electra_ipu.json')

In [8]:
ipu_model = PipelinedBertForSequenceClassification.from_transformers(model, ipu_config)


In [9]:
ipu_model.parallelize()
if not training_args.fp32:
    ipu_model = ipu_model.half()

# GLUE datatest

In [10]:
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
)

In [11]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [12]:
import datasets
import numpy as np
from datasets import load_dataset, load_metric


In [13]:
#raw_datasets = load_dataset("glue", 'mnli', cache_dir=model_args.cache_dir)


In [14]:
# label_list = raw_datasets["train"].unique("label")
# label_list.sort()  # Let's sort it for determinism
# num_labels = len(label_list)

# sentence1_key, sentence2_key = task_to_keys['mnli']
# padding=False
# max_seq_length = tokenizer.model_max_length
# label_to_id = PretrainedConfig(num_labels=num_labels).label2id

In [15]:
def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    )
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = examples["label"]
        #result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result

# with training_args.main_process_first(desc="dataset map pre-processing"):
#     raw_datasets = raw_datasets.map(
#         preprocess_function,
#         batched=True,
#         load_from_cache_file=not data_args.overwrite_cache,
#         desc="Running tokenizer on dataset",
#     )

In [16]:
# train_dataset = raw_datasets['train']

# import inspect

# signature = inspect.signature(training_model.forward)
# _signature_columns = list(signature.parameters.keys())
# _signature_columns += ["label", "label_ids"]

# columns = [k for k in _signature_columns if k in train_dataset.column_names]
# ignored_columns = list(set(train_dataset.column_names) - set(_signature_columns))

# train_dataset = train_dataset.remove_columns(ignored_columns)

In [17]:
import poptorch

In [18]:
opts = ipu_config.to_options()
optimizer_class = torch.optim.AdamW
optimizer = optimizer_class(model.parameters(), lr=0.004)

Setting replicated_tensor_sharding to False when replication_factor=1


In [19]:
opts.deviceIterations(4)

In [20]:
training_model = poptorch.trainingModel(
    ipu_model.train(), options=opts, optimizer=optimizer
)

In [21]:
#model(**{'input_ids':torch.tensor([data['input_ids']]), 'token_type_ids':torch.tensor([data['token_type_ids']]), 'attention_mask':torch.tensor([data['attention_mask']])})

In [24]:
#training_model.compile(**{'input_ids':torch.tensor([data['input_ids']]), 'token_type_ids':torch.tensor([data['token_type_ids']]), 'attention_mask':torch.tensor([data['attention_mask']])})

In [25]:
inputss = tokenizer(["Hello, my dog is cute"]*4, return_tensors="pt")

In [ ]:
training_model.compile(**dict(inputss))


In [59]:
inputss

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102],
        [  101,  7592,  1010,  2026,  3899,  2003, 10140,   102],
        [  101,  7592,  1010,  2026,  3899,  2003, 10140,   102],
        [  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
inference_model = poptorch.inferenceModel(
    ipu_model.eval(), options=opts
)

In [65]:
inference_model(**dict(inputss))

(tensor([[-0.0570,  0.0084,  0.0037]], dtype=torch.float16),)

In [64]:
#inference_model(**{'input_ids':torch.tensor([data['input_ids']]), 'token_type_ids':torch.tensor([data['token_type_ids']]), 'attention_mask':torch.tensor([data['attention_mask']])})